In [ ]:
from network_definitions import VGG16 # choose the model you want

In [ ]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torchvision
from torch.autograd import Variable
import torch.optim as optim
from torchvision import transforms

from models.conv2model import *

parser = argparse.ArgumentParser(description='PyTorch CIFAR PGD Attack Evaluation')
parser.add_argument("-f", "--fff", help="a dummy argument to fool ipython", default="1")
parser.add_argument('--test-batch-size', type=int, default=100, metavar='N',
                    help='input batch size for testing (default: 200)')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='disables CUDA training')
parser.add_argument('--epsilon', default=0.031,
                    help='perturbation')
parser.add_argument('--num-steps', default=20,
                    help='perturb number of steps')
parser.add_argument('--step-size', default=0.003,
                    help='perturb step size')
parser.add_argument('--random',
                    default=True,
                    help='random initialization for PGD')
parser.add_argument('--model-path',
                    default='./models/VGG16-BASE-nnmodule_withdropoutmodel_150_91.pth',
                    help='model for white-box attack evaluation')
parser.add_argument('--source-model-path',
                    default='./models/conv2_model_80_74.pth',
                    help='source model for black-box attack evaluation')
parser.add_argument('--target-model-path',
                    default='./models/VGG16-BASE-nnmodule_withdropoutmodel_150_91.pth',
                    help='target model for black-box attack evaluation')
parser.add_argument('--white-box-attack', default=False,
                    help='whether perform white-box attack')

args = parser.parse_args()

# settings
use_cuda = not args.no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

# set up data loader
mean_cifar10 = [0.485, 0.456, 0.406]   
std_cifar10 = [0.229, 0.224, 0.225]
batch_size = 1

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean_cifar10,std_cifar10),
])

testset = torchvision.datasets.CIFAR10(root='../FlexibleCNNs/data', train=False, download=True, transform=transform_test)
test_loader = torch.utils.data.DataLoader(testset, batch_size=args.test_batch_size, shuffle=False, **kwargs)


def _pgd_whitebox(model,
                  X,
                  y,
                  epsilon=args.epsilon,
                  num_steps=args.num_steps,
                  step_size=args.step_size):
    out = model(X)
    err = (out.data.max(1)[1] != y.data).float().sum()
    X_pgd = Variable(X.data, requires_grad=True)
    if args.random:
        random_noise = torch.FloatTensor(*X_pgd.shape).uniform_(-epsilon, epsilon).to(device)
        X_pgd = Variable(X_pgd.data + random_noise, requires_grad=True)

    for _ in range(num_steps):
        opt = optim.SGD([X_pgd], lr=1e-3)
        opt.zero_grad()

        with torch.enable_grad():
            loss = nn.CrossEntropyLoss()(model(X_pgd), y)
        loss.backward()
        eta = step_size * X_pgd.grad.data.sign()
        X_pgd = Variable(X_pgd.data + eta, requires_grad=True)
        eta = torch.clamp(X_pgd.data - X.data, -epsilon, epsilon)
        X_pgd = Variable(X.data + eta, requires_grad=True)
        X_pgd = Variable(torch.clamp(X_pgd, 0, 1.0), requires_grad=True)
    err_pgd = (model(X_pgd).data.max(1)[1] != y.data).float().sum()
    print('err pgd (white-box): ', err_pgd.item())
    return err, err_pgd


def _pgd_blackbox(model_target,
                  model_source,
                  X,
                  y,
                  epsilon=args.epsilon,
                  num_steps=args.num_steps,
                  step_size=args.step_size):
    out = model_target(X)
    err = (out.data.max(1)[1] != y.data).float().sum()
    X_pgd = Variable(X.data, requires_grad=True)
    if args.random:
        random_noise = torch.FloatTensor(*X_pgd.shape).uniform_(-epsilon, epsilon).to(device)
        X_pgd = Variable(X_pgd.data + random_noise, requires_grad=True)

    for _ in range(num_steps):
        opt = optim.SGD([X_pgd], lr=1e-3)
        opt.zero_grad()
        with torch.enable_grad():
            loss = nn.CrossEntropyLoss()(model_source(X_pgd), y)
        loss.backward()
        eta = step_size * X_pgd.grad.data.sign()
        X_pgd = Variable(X_pgd.data + eta, requires_grad=True)
        eta = torch.clamp(X_pgd.data - X.data, -epsilon, epsilon)
        X_pgd = Variable(X.data + eta, requires_grad=True)
        X_pgd = Variable(torch.clamp(X_pgd, 0, 1.0), requires_grad=True)

    err_pgd = (model_target(X_pgd).data.max(1)[1] != y.data).float().sum()
    #print('err pgd black-box: ', err_pgd.item())
    return err, err_pgd


def eval_adv_test_whitebox(model, device, test_loader):
    """
    evaluate model by white-box attack
    """
    model.eval()
    robust_err_total = 0
    natural_err_total = 0

    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        # pgd attack
        X, y = Variable(data, requires_grad=True), Variable(target)
        err_natural, err_robust = _pgd_whitebox(model, X, y)
        robust_err_total += err_robust
        natural_err_total += err_natural
    print('natural_err_total: ', natural_err_total)
    print('robust_err_total: ', robust_err_total)


def eval_adv_test_blackbox(model_target, model_source, device, test_loader):
    """
    evaluate model by black-box attack
    """
    model_target.eval()
    model_source.eval()
    robust_err_total = 0
    natural_err_total = 0

    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        # pgd attack
        X, y = Variable(data, requires_grad=True), Variable(target)
        err_natural, err_robust = _pgd_blackbox(model_target, model_source, X, y)
        robust_err_total += err_robust
        natural_err_total += err_natural
    print('natural_err_total: ', natural_err_total)
    print('robust_err_total: ', robust_err_total)
    
    return natural_err_total, robust_err_total


In [ ]:
# Main
pgd_naterror = []
pgd_roberror = []

for i in range(100): #do 100 iterations then find average robustness
    if args.white_box_attack:
        # white-box attack
        print('pgd white-box attack:', i)
        model = VGG16().to(device)
        # Load the pretrained model
        state = torch.load('./models/VGG16-BASE-nnmodule_withdropoutmodel_150_91.pth')
        model.load_state_dict(state['model'])

        eval_adv_test_whitebox(model, device, test_loader)
    else:
        # black-box attack
        print('pgd black-box attack:', i)
        model_target = VGG16().to(device)
        state_target = torch.load(args.target_model_path)
        model_target.load_state_dict(state_target['model'])
        model_source = conv2model().to(device)
        state_source = torch.load(args.source_model_path)
        model_source.load_state_dict(state_source['model'])

        nat_err, rob_err = eval_adv_test_blackbox(model_target, model_source, device, test_loader)
        pgd_naterror.append(nat_err)
        pgd_roberror.append(rob_err)
        

In [ ]:
# Calculate average Accuracy
# Accuracy = 10,000 - error.
# It is 10,000 because batch size is 100 images and each image is a %, so 100x100 = 10,000

avg_naterror = sum(pgd_naterror)/ len(pgd_naterror)
avg_natacc = (10000 - avg_naterror)/100
print("Average Nat. Acc = ", avg_natacc)

avg_roberror = sum(pgd_roberror)/ len(pgd_roberror)
avg_robacc = (10000 - avg_roberror)/100
print("Average Rob. Acc = ", avg_robacc)


In [ ]:
#Save
import pickle
with open("pgd_blackbox_base_avgrobacc.pkl", "wb") as output_file:
        pickle.dump(avg_robacc, output_file)